In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import timm
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


CONFIG = {
    'image_dir': '/kaggle/input/notebooks/kagglertw/shemagh-right-place/images',
    'csv_path': '/kaggle/input/notebooks/kagglertw/shemagh-right-place/train_df.csv',
    'img_size': 240,
    'batch_size': 32,
    'num_epochs': 25,  
    'learning_rate': 5e-5,  
    'weight_decay': 1e-4,  
    'num_folds': 5,
    'seed': 42,
    'num_workers': 2,
    'model_name': 'efficientnet_b0',
    'save_dir': './weights_improved',
    'early_stopping_patience': 5,  
    'use_class_weights': True,  
    'dropout': 0.4,  
}

os.makedirs(CONFIG['save_dir'], exist_ok=True)

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(CONFIG['seed'])


class BinaryImageDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = self.df.loc[idx, 'filename']
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        label = self.df.loc[idx, 'right_place']
        return image, torch.tensor(label, dtype=torch.float32)


train_transform = transforms.Compose([
    transforms.Resize((CONFIG['img_size'], CONFIG['img_size'])),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.2),  
    transforms.RandomRotation(20),  
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.15),  
    transforms.RandomGrayscale(p=0.1),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.15)),  
])

val_transform = transforms.Compose([
    transforms.Resize((CONFIG['img_size'], CONFIG['img_size'])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])


class EfficientNetB1Classifier(nn.Module):
    def __init__(self, pretrained=True, dropout=0.4):
        super(EfficientNetB1Classifier, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=pretrained)
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(in_features, 1)
        )
        
    def forward(self, x):
        return self.model(x)


class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001, mode='max'):
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        
    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
        elif self.mode == 'max':
            if score < self.best_score + self.min_delta:
                self.counter += 1
                if self.counter >= self.patience:
                    self.early_stop = True
            else:
                self.best_score = score
                self.counter = 0
        elif self.mode == 'min':
            if score > self.best_score - self.min_delta:
                self.counter += 1
                if self.counter >= self.patience:
                    self.early_stop = True
            else:
                self.best_score = score
                self.counter = 0

def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    predictions = []
    targets = []
    
    pbar = tqdm(loader, desc='Training')
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        
        preds = torch.sigmoid(outputs).detach().cpu().numpy()
        predictions.extend(preds)
        targets.extend(labels.cpu().numpy())
        
        pbar.set_postfix({'loss': loss.item()})
    
    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss, np.array(predictions), np.array(targets)

def validate_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    predictions = []
    targets = []
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Validation')
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            
            preds = torch.sigmoid(outputs).cpu().numpy()
            predictions.extend(preds)
            targets.extend(labels.cpu().numpy())
            
            pbar.set_postfix({'loss': loss.item()})
    
    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss, np.array(predictions), np.array(targets)

def calculate_metrics(predictions, targets, threshold=0.5):
    pred_binary = (predictions >= threshold).astype(int)
    accuracy = accuracy_score(targets, pred_binary)
    auc = roc_auc_score(targets, predictions)
    f1 = f1_score(targets, pred_binary)
    return accuracy, auc, f1

def train_fold(fold, train_df, val_df, config):
    print(f"\n{'='*50}")
    print(f"Training Fold {fold + 1}/{config['num_folds']}")
    print(f"{'='*50}")
    
    
    train_dataset = BinaryImageDataset(train_df, config['image_dir'], train_transform)
    val_dataset = BinaryImageDataset(val_df, config['image_dir'], val_transform)
    
    
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], 
                            shuffle=True, num_workers=config['num_workers'])
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], 
                          shuffle=False, num_workers=config['num_workers'])
    
    
    model = EfficientNetB1Classifier(
        pretrained=True, 
        dropout=config['dropout']
    ).to(device)
    
    
    if config['use_class_weights']:
        pos_count = (train_df['right_place'] == 1).sum()
        neg_count = (train_df['right_place'] == 0).sum()
        pos_weight = torch.tensor([neg_count / pos_count]).to(device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        print(f"Using weighted loss - Pos weight: {pos_weight.item():.2f}")
    else:
        criterion = nn.BCEWithLogitsLoss()
    
    
    optimizer = optim.AdamW(
        model.parameters(), 
        lr=config['learning_rate'],
        weight_decay=config['weight_decay']
    )
    
    
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=5, T_mult=1, eta_min=1e-7
    )
    
    
    early_stopping = EarlyStopping(
        patience=config['early_stopping_patience'],
        mode='max'
    )
    
    best_val_loss = float('inf')
    best_auc = 0.0
    
    
    for epoch in range(config['num_epochs']):
        print(f"\nEpoch {epoch + 1}/{config['num_epochs']} | LR: {optimizer.param_groups[0]['lr']:.2e}")
        
        
        train_loss, train_preds, train_targets = train_epoch(
            model, train_loader, criterion, optimizer, device
        )
        train_acc, train_auc, train_f1 = calculate_metrics(train_preds, train_targets)
        
        
        val_loss, val_preds, val_targets = validate_epoch(
            model, val_loader, criterion, device
        )
        val_acc, val_auc, val_f1 = calculate_metrics(val_preds, val_targets)
        
        
        scheduler.step()
        
        print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | AUC: {train_auc:.4f} | F1: {train_f1:.4f}")
        print(f"Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | AUC: {val_auc:.4f} | F1: {val_f1:.4f}")
        
        
        if val_auc > best_auc:
            best_auc = val_auc
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'val_auc': val_auc,
                'val_acc': val_acc,
                'config': config,
            }, os.path.join(config['save_dir'], f'effnet_b0_fold{fold+1}_best.pth'))
            print(f"✓ Saved best model (AUC: {best_auc:.4f})")
        
        
        early_stopping(val_auc)
        if early_stopping.early_stop:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break
    
    return best_val_loss, best_auc

def main():
    print("Loading data...")
    df = pd.read_csv(CONFIG['csv_path'])
    print(f"Dataset shape: {df.shape}")
    print(f"Class distribution:\n{df['right_place'].value_counts()}")
    print(f"Class imbalance ratio: {(df['right_place']==0).sum() / (df['right_place']==1).sum():.2f}:1")
    
    
    skf = StratifiedKFold(n_splits=CONFIG['num_folds'], shuffle=True, random_state=CONFIG['seed'])
    
    
    fold_results = []
    
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['right_place'])):
        train_df = df.iloc[train_idx]
        val_df = df.iloc[val_idx]
        
        print(f"\nFold {fold + 1} - Train: {len(train_df)}, Val: {len(val_df)}")
        print(f"Train class distribution:\n{train_df['right_place'].value_counts()}")
        print(f"Val class distribution:\n{val_df['right_place'].value_counts()}")
        
        val_loss, val_auc = train_fold(fold, train_df, val_df, CONFIG)
        fold_results.append({'fold': fold + 1, 'val_loss': val_loss, 'val_auc': val_auc})
    
    
    print(f"\n{'='*50}")
    print("Cross-Validation Summary")
    print(f"{'='*50}")
    results_df = pd.DataFrame(fold_results)
    print(results_df)
    print(f"\nMean AUC: {results_df['val_auc'].mean():.4f} ± {results_df['val_auc'].std():.4f}")
    print(f"Mean Loss: {results_df['val_loss'].mean():.4f} ± {results_df['val_loss'].std():.4f}")
    
    
    results_df.to_csv(os.path.join(CONFIG['save_dir'], 'cv_results.csv'), index=False)
    print(f"\n✓ Results saved to {CONFIG['save_dir']}/cv_results.csv")
    print(f"✓ Model weights saved in {CONFIG['save_dir']}/")


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Using device: cuda


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
from tqdm import tqdm


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


class EfficientNetB1Classifier(nn.Module):
    def __init__(self, pretrained=False):
        super(EfficientNetB1Classifier, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=pretrained)
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(in_features, 1)
        )
        
    def forward(self, x):
        return self.model(x)


class InferenceDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image


def predict(model, loader, device):
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for images in tqdm(loader, desc='Predicting'):
            images = images.to(device)
            outputs = model(images).squeeze()
            preds = torch.sigmoid(outputs).cpu().numpy()
            
            if preds.ndim == 0:  
                predictions.append(float(preds))
            else:
                predictions.extend(preds.tolist())
    
    return np.array(predictions)


def ensemble_predict(weights_dir, image_paths, img_size=240, batch_size=32, num_folds=5):
    
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    
    dataset = InferenceDataset(image_paths, transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    all_predictions = []
    
    
    for fold in range(num_folds):
        weight_path = os.path.join(weights_dir, f'effnet_b0_fold{fold+1}_best.pth')
        
        if not os.path.exists(weight_path):
            print(f"Warning: {weight_path} not found, skipping...")
            continue
        
        print(f"\nLoading fold {fold + 1} weights...")
        model = EfficientNetB1Classifier(pretrained=False).to(device)
        checkpoint = torch.load(
    weight_path,
    map_location=device,
    weights_only=False   
)
        model.load_state_dict(checkpoint['model_state_dict'])
        
        print(f"Fold {fold + 1} - Val AUC: {checkpoint.get('val_auc', 'N/A'):.4f}")
        
        predictions = predict(model, loader, device)
        all_predictions.append(predictions)
        
        del model
        torch.cuda.empty_cache()
    
    
    ensemble_preds = np.mean(all_predictions, axis=0)
    
    return ensemble_preds, all_predictions


def single_model_predict(weight_path, image_paths, img_size=240, batch_size=32):
    
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    
    dataset = InferenceDataset(image_paths, transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    
    print(f"Loading model from {weight_path}...")
    model = EfficientNetB1Classifier(pretrained=False).to(device)
    checkpoint = torch.load(
    weight_path,
    map_location=device,
    weights_only=False   
)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"Model Val AUC: {checkpoint.get('val_auc', 'N/A'):.4f}")
    
    
    predictions = predict(model, loader, device)
    
    return predictions


if __name__ == '__main__':
    
    image_dir = '/kaggle/input/datasets/kagglertw/dal-shemagh/dal-shemagh-detection-challenge/images/test'
    weights_dir = '/kaggle/input/notebooks/kagglertw/shemagh-right-place-binary/weights_improved/'
    
    
    test_csv = 'test.csv'  
    
    
    if os.path.exists(test_csv):
        test_df = pd.read_csv(test_csv)
        image_paths = [os.path.join(image_dir, fname) for fname in test_df['filename']]
    else:
        
        image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) 
                      if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    print(f"Found {len(image_paths)} images")
    
    
    print("\n" + "="*50)
    print("Running Ensemble Prediction")
    print("="*50)
    ensemble_preds, fold_preds = ensemble_predict(weights_dir, image_paths, num_folds=5)
    
    
    results_df = pd.DataFrame({
        'filename': [os.path.basename(p) for p in image_paths],
        'prediction_prob': ensemble_preds,
        'prediction_class': (ensemble_preds >= 0.5).astype(int)
    })
    
    
    for i, preds in enumerate(fold_preds):
        results_df[f'fold_{i+1}_prob'] = preds
    
    
    results_df.to_csv('predictions.csv', index=False)
    print(f"\n✓ Predictions saved to predictions.csv")
    print(f"\nSample predictions:")
    print(results_df.head(10))
    
    print(f"\nPrediction distribution:")
    print(results_df['prediction_class'].value_counts())

Using device: cuda
Found 842 images

Running Ensemble Prediction

Loading fold 1 weights...
Fold 1 - Val AUC: 0.8206


Predicting: 100%|██████████| 27/27 [00:10<00:00,  2.62it/s]



Loading fold 2 weights...
Fold 2 - Val AUC: 0.8363


Predicting: 100%|██████████| 27/27 [00:04<00:00,  6.02it/s]



Loading fold 3 weights...
Fold 3 - Val AUC: 0.8844


...


Loading fold 4 weights...
Fold 4 - Val AUC: 0.8487


Predicting: 100%|██████████| 27/27 [00:04<00:00,  6.37it/s]



Loading fold 5 weights...
Fold 5 - Val AUC: 0.7672


Predicting: 100%|██████████| 27/27 [00:04<00:00,  6.22it/s]


✓ Predictions saved to predictions.csv

Sample predictions:
  filename  prediction_prob  prediction_class  fold_1_prob  fold_2_prob  \
0  623.jpg         0.750885                 1     0.830798     0.756628   
1  764.jpg         0.248612                 0     0.159409     0.349164   
2  771.jpg         0.519455                 1     0.554730     0.541493   
3  208.jpg         0.395363                 0     0.377498     0.405647   
4  820.jpg         0.552467                 1     0.609143     0.526818   
5  473.jpg         0.214795                 0     0.200305     0.152140   
6  333.jpg         0.547931                 1     0.567810     0.563017   
7  537.jpg         0.212417                 0     0.120150     0.261906   
8   45.jpg         0.438628                 0     0.459107     0.479885   
9  369.jpg         0.220761                 0     0.095426     0.311770   

   fold_3_prob  fold_4_prob  fold_5_prob  
0     0.724820     0.663660     0.778519  
1     0.259275     0.348190